# Create SparkContext & SparkSession

https://spark.apache.org/docs/latest/rdd-programming-guide.html

http://spark.apache.org/docs/latest/sql-getting-started.html

In [1]:
import findspark
findspark.init()

import pyspark
sc = pyspark.SparkContext(appName='jupyter')

from pyspark.sql import SparkSession, Row
se = SparkSession(sc)

# Yandex.Music dataset

In [1]:
# download from s3 to local disk
! aws s3 cp s3://ydatazian/yandex_music ./yandex_music --recursive

download: s3://ydatazian/yandex_music/README.txt to yandex_music/README.txt
download: s3://ydatazian/yandex_music/artists.jsonl to yandex_music/artists.jsonl
download: s3://ydatazian/yandex_music/events.csv to yandex_music/events.csv


In [2]:
! ls -lh yandex_music

total 52M
-rw-r--r-- 1 jovyan root 3.7M Oct 19 00:18 artists.jsonl
-rw-r--r-- 1 jovyan root  48M Oct 19 00:28 events.csv
-rw-r--r-- 1 jovyan root  254 Oct 19 00:41 README.txt


In [3]:
! head -n 5 yandex_music/artists.jsonl

{"artistId":0,"artistName":"Mack Gordon"}
{"artistId":1,"artistName":"Kenny Dorham"}
{"artistId":2,"artistName":"Max Roach"}
{"artistId":3,"artistName":"Francis Rossi"}
{"artistId":4,"artistName":"Status Quo"}


In [4]:
! head -n 5 yandex_music/events.csv

userId,artistId,plays,skips
0,335,1,0
0,708,1,0
0,710,2,1
0,815,1,1


# Copy to HDFS

In [5]:
! hadoop fs -copyFromLocal yandex_music /

In [6]:
! hadoop fs -ls -h /yandex_music

Found 3 items
-rw-r--r--   1 jovyan supergroup        254 2020-10-24 16:50 /yandex_music/README.txt
-rw-r--r--   1 jovyan supergroup      3.7 M 2020-10-24 16:50 /yandex_music/artists.jsonl
-rw-r--r--   1 jovyan supergroup     47.6 M 2020-10-24 16:50 /yandex_music/events.csv


# Load to Spark

In [7]:
artists = se.read.json("hdfs:///yandex_music/artists.jsonl")
artists.registerTempTable("artists")
artists.limit(5).toPandas()

,artistId,artistName
0,0,Mack Gordon
1,1,Kenny Dorham
2,2,Max Roach
3,3,Francis Rossi
4,4,Status Quo


In [8]:
events = se.read.csv("hdfs:///yandex_music/events.csv", header=True, 
                     schema='userId bigint, artistId bigint, plays INT, skips INT')
events.registerTempTable("events")
events.limit(5).toPandas()

,userId,artistId,plays,skips
0,0,335,1,0
1,0,708,1,0
2,0,710,2,1
3,0,815,1,1
4,0,880,1,1


In [4]:
# stats
se.sql("""
select
    count(distinct userId) as users,
    count(distinct artistId) as artists,
    count(*) as interactions,
    count(*) / (count(distinct userId) * count(distinct artistId)) as density
from 
    events
""").toPandas()

,users,artists,interactions,density
0,4999,53031,3412504,0.012872


In [ ]:
# most popular artists
se.sql("""
select
    artists.artistName,
    sum(plays) as popularity
from 
    events join artists on events.artistId = artists.artistId
group by artistName
order by popularity desc
limit 30
""").toPandas()

# Train iALS

rating = plays

In [6]:
%%time
train, test = events.rdd.randomSplit([0.95, 0.05], seed=0)

# turn on caching for speed
train.cache()
test.cache()

# populate cache
train.count()
test.count()

CPU times: user 14.7 ms, sys: 8.57 ms, total: 23.3 ms
Wall time: 27.7 s


In [7]:
train.take(5)

[Row(userId=0, artistId=335, plays=1, skips=0),
 Row(userId=0, artistId=708, plays=1, skips=0),
 Row(userId=0, artistId=710, plays=2, skips=1),
 Row(userId=0, artistId=815, plays=1, skips=1),
 Row(userId=0, artistId=880, plays=1, skips=1)]

In [24]:
%%time
from pyspark.mllib.recommendation import ALS
model = ALS().trainImplicit(
    train.map(lambda x: (x.userId, x.artistId, np.log2(x.plays + 1))),
    rank=32, iterations=10, lambda_=0.01, alpha=10.0, seed=0
)

CPU times: user 95.6 ms, sys: 22.6 ms, total: 118 ms
Wall time: 1min


In [25]:
# artists' embeddings
import numpy as np

artist_to_name = {}
for row in artists.collect():
    artist_to_name[row.artistId] = row.artistName

artist_ids = []
artist_names = []
artist_profiles = []

for artistId, profile in model.productFeatures().collect():
    artist_ids.append(artistId)
    artist_names.append(artist_to_name[artistId])
    artist_profiles.append(profile)

artist_ids = np.array(artist_ids)
artist_names = np.array(artist_names)
artist_profiles = np.vstack(artist_profiles)
print(artist_profiles.shape)

(52665, 32)


# Similar artists

In [26]:
target_artists = {index: v 
                  for index, v in enumerate(artist_names) 
                  if "Coldplay" == v or "50 Cent" == v or "AC/DC" == v}
target_artists

{11563: '50 Cent', 22207: 'AC/DC', 32914: 'Coldplay'}

In [27]:
import scipy
import scipy.spatial

for index, name in target_artists.items():
    print("#############", name, "#############")
    
    cosines = (-scipy.spatial.distance.cdist([artist_profiles[index]], artist_profiles, metric='cosine') + 1)[0]
    cosines[np.isnan(cosines)] = -1e20

    for idx in np.argsort(cosines)[::-1][:10]:
        print(artist_names[idx], "\t", cosines[idx])

############# 50 Cent #############
50 Cent 	 1.0
Dr. Dre 	 0.87176920263
Lloyd Banks 	 0.862571081123
Jay-Z 	 0.835235793381
2Chainz 	 0.822563140385
Cashis 	 0.812456739525
Snoop Dogg 	 0.810484217851
Missy  Elliott 	 0.805635315489
Akon 	 0.798163079104
Busta Rhymes 	 0.787780080328
############# AC/DC #############
AC/DC 	 1.0
The Offspring 	 0.880806285576
Nirvana 	 0.87660720058
Metallica 	 0.874860631399
Red Hot Chili Peppers 	 0.873863202133
System of A Down 	 0.853683802014
Limp Bizkit 	 0.853453742095
Bon Jovi 	 0.850678176659
Nickelback 	 0.846709868065
Scorpions 	 0.842406452006
############# Coldplay #############
Coldplay 	 1.0
Lana Del Rey 	 0.957398977112
Adele 	 0.954771689577
OneRepublic 	 0.951114667239
Maroon 5 	 0.950856957375
Sam Smith 	 0.929694792036
Katy Perry 	 0.921320013598
Ed Sheeran 	 0.919972991344
Pharrell Williams 	 0.919839118559
Twenty One Pilots 	 0.91862092206


# NDCG

In [12]:
def dcg(ratings):
    return float(np.sum((2 ** np.array(ratings, np.float32) - 1) / np.log2(np.arange(1, len(ratings) + 1) + 1)))


def ndcg(ratings, at=None):
    idcg = dcg(sorted(ratings, reverse=True))
    return dcg(ratings) / idcg if idcg > 0 else 0


def ndcg_score(y_true, y_pred):
    assert y_true.shape == y_pred.shape
    order = np.argsort(y_pred)[::-1]
    return ndcg(y_true[order])


# tests
def test1():
    y_true = np.array([  0,   0,   2,   1,   0])
    y_pred = np.array([0.2, 0.1, 0.5, 0.3, 0.4])
    correct_ndcg = (3 / np.log(1 + 1) + 1 / np.log(3 + 1)) / (3 / np.log(1 + 1) + 1 / np.log(2 + 1))
    assert np.allclose(ndcg_score(y_true, y_pred), correct_ndcg)

    
def test2():
    y_true = np.array([  0,   0,   0,   0,   0])
    y_pred = np.array([0.2, 0.1, 0.5, 0.3, 0.4])
    assert np.allclose(ndcg_score(y_true, y_pred), 0.0)

    
def test3():
    y_true = np.array([  1,   0,   0,   0,   0])
    y_pred = np.array([0.2, 0.1, 0.5, 0.3, 0.4])
    correct_ndcg = (1 / np.log(4 + 1)) / (1 / np.log(1 + 1))
    assert np.allclose(ndcg_score(y_true, y_pred), correct_ndcg)


test1()
test2()
test3()

In [13]:
print(dcg([5, 4, 3, 2, 1]))
print(dcg([3, 4, 5, 2, 1]))
print(dcg([5, 4, 1, 2, 3]))

45.64282878502658
33.64282878502658
44.963945628433834


# NDCG of baseline

Rank artists by global popularity

In [14]:
artist_to_popularity = (
    train
    .map(lambda x: (x.artistId, x.plays))
    .reduceByKey(lambda a, b: a + b)
    .collect()
)

artist_to_popularity = {a: p for a, p in artist_to_popularity}

In [15]:
predictions_and_ratings_per_user = (
    test
    .map(lambda x: (x.userId, (artist_to_popularity.get(x.artistId, 0), np.log2(x.plays + 1))))
    .groupByKey()
    .map(lambda x: (x[0], list(x[1])))
)

In [16]:
predictions_and_ratings_per_user.take(1)

[(0,
  [(2371, 1.0),
   (22102, 3.3219280948873622),
   (609, 1.0),
   (7399, 1.0),
   (884, 1.5849625007211561),
   (481, 1.5849625007211561),
   (4008, 1.0),
   (2325, 1.5849625007211561),
   (774, 1.0),
   (1035, 1.5849625007211561),
   (4484, 3.5849625007211561),
   (234, 1.0),
   (1523, 1.0),
   (2273, 1.5849625007211561),
   (1243, 2.3219280948873622),
   (5388, 2.3219280948873622),
   (7856, 5.9772799234999168),
   (781, 1.0),
   (4743, 1.5849625007211561),
   (1234, 1.0),
   (1569, 0.0),
   (650, 0.0),
   (817, 0.0),
   (1579, 0.0),
   (1397, 0.0),
   (9438, 0.0),
   (9, 0.0),
   (773, 0.0),
   (543, 0.0),
   (46, 0.0),
   (574, 0.0),
   (9578, 0.0),
   (241, 0.0)])]

In [17]:
def ndcg_for_user(x):
    y_pred = np.array([e[0] for e in x])
    y_true = np.array([e[1] for e in x])
    return ndcg_score(y_true, y_pred)
    
(
    predictions_and_ratings_per_user
    .map(lambda x: ndcg_for_user(x[1]))
    .mean()
)

0.6610135790248743

# NDCG of iALS

In [28]:
predictions = (
    model
    .predictAll(test.map(lambda x: (x.userId, x.artistId)))
    .map(lambda x: ((x[0], x[1]), x[2]))
)

In [29]:
predictions.take(5)

[((3949, 17312), 0.8030995046830238),
 ((2464, 17312), 0.7113483998861412),
 ((1858, 17312), 0.3888120783910832),
 ((4147, 17312), -0.1829481711739028),
 ((77, 3456), 0.30611075912876945)]

In [30]:
predictions_and_ratings_per_user = (
    predictions
    .join(test.map(lambda x: ((x.userId, x.artistId), np.log2(x.plays + 1))))
    .map(lambda x: (x[0][0], x[1]))
    .groupByKey()
    .map(lambda x: (x[0], list(x[1])))
)

In [31]:
predictions_and_ratings_per_user.take(1)

[(2816,
  [(0.2346505020090635, 0.0),
   (0.6805648268080089, 1.0),
   (0.002171185035217969, 0.0),
   (0.7116594521569385, 0.0),
   (0.6533145646762915, 0.0),
   (0.8557148872377391, 5.7548875021634682),
   (0.5048028191164565, 1.5849625007211561),
   (0.14434127345083803, 0.0),
   (1.001120439472722, 2.3219280948873622),
   (0.9308679654406266, 3.0),
   (0.42996150182789633, 0.0),
   (0.3494304392722968, 0.0),
   (0.6577348212113873, 0.0),
   (0.8763885448939661, 1.0),
   (0.5544562197707432, 0.0),
   (0.47719639061737673, 1.0),
   (0.9753262163922594, 3.8073549220576042),
   (0.598020939013476, 1.0),
   (0.8114790918340737, 0.0),
   (0.5809819236480271, 0.0),
   (0.01893891198655006, 3.9068905956085187),
   (0.7632991406863413, 0.0),
   (-0.10761971593888606, 0.0),
   (0.9885897265248467, 0.0),
   (0.4867501410680902, 0.0),
   (0.38156062745105784, 0.0),
   (0.46353917334755157, 1.0),
   (0.5385095386026597, 0.0),
   (0.44012027262057335, 1.0),
   (0.39191343700735914, 0.0),
   (0.6

In [32]:
def ndcg_for_user(x):
    y_pred = np.array([e[0] for e in x])
    y_true = np.array([e[1] for e in x])
    return ndcg_score(y_true, y_pred)
    
(
    predictions_and_ratings_per_user
    .map(lambda x: ndcg_for_user(x[1]))
    .mean()
)

0.7165279297105116

In [ ]:
print("Improved by {:0.3} percent!".format(100 * (0.716 / 0.661 - 1)))

# Homework

On the lecture we've discussed Jaccard similarity and how to compute it using MapReduce.

Use Spark to answer the following questions:
1. **(1 point) How many pairs of artists have non-zero Jaccard similarity (consider all of them, even (a, a)).**
2. **(1 point) Find 5 most similar artists to "Maroon 5" using Jaccard similarity.**

Hints:
- Use data, loaded <a href="#Load-to-Spark">here</a>.
- To reduce the complexity of Jaccard similarity computation let's consider only entries with $plays > 2$.
- For more confident Jaccard estimates let's consider artists who have more than 50 users in the filtered dataset.
- For debugging please use <a href="https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.sample">events.sample(False, 0.01)</a> to reduce computation time.
- Let's say that statistics (like popularity) of all artists fit in memory.
- Play around with parallelism of operations, e.g. 
<a href="https://spark.apache.org/docs/latest/api/python/pyspark.html#pyspark.RDD.groupByKey">groupByKey(numPartitions=100)</a>, to see more precise progress bar.
- You can store intermidiate results on HDFS if you need.
- Please be patient, computation might take tens of minutes.
- You can use Spark SQL if you like.

In [68]:
events.show(1)

+------+--------+-----+-----+
|userId|artistId|plays|skips|
+------+--------+-----+-----+
|     0|     335|    1|    0|
+------+--------+-----+-----+
only showing top 1 row



In [69]:
se.sql(
"""
select 
    userId,
    count(*) as cnt
from events
where plays > 2
group by userId
order by cnt desc
""").show(5)

+------+---+
|userId|cnt|
+------+---+
|  4689|644|
|  3121|577|
|  4575|572|
|  2266|519|
|  4217|512|
+------+---+
only showing top 5 rows



In [ ]:
# YOUR CODE HERE

In [ ]:
# sc.stop()